## Planificación de la producción: frutería

V1.0, Julio 23

#### Problema CQM c/ reformulación Qubo

En este interesante e intuitivo problema CQM se modela el beneficio de una frutería atendiendo a la producción de dos surtidos de frutas.

La función objetivo codifica la diferencia entre el importe de venta de los lotes y el coste de la materia prima usada para su confección.

Una frutería vende dos tipos de surtidos de frutos rojos, A y B. 


- El surtido de tipo A contiene 75 g de arándanos y 100 g de frambuesas, se vende a 3 euros

- El surtido de tipo B contiene 75 g de arándanos y 50 g de frambuesas, se vende a  2 euros. 

- La frutería dispone de un total de 4 kg de arándanos y 5 kg de frambuesas 

- Las ventas de tipo A, siempre es menor o igual al doble de los de tipo B.


Para el frutero el coste de las materias primas son:

- 500g de frambuesa le cuestan 6€, y de arándano 8€

<b> Se pide </b>

1. Determinar los lotes que deben de prepararse de cada clase para maximar importe ventas

2. Deducir beneficio:

    - manualmente, a partir del dimensionado de los lotes
    - codificando el coste en la función objetivo.


<b>Planteamiento algebraico:</b>

    - x1, x2 ; variables  A y  B

    - Función objetivo 1: minimizar{f(x1,x2)} = -3*x1 - 2*x2
    - Función objetivo 2: minimizar {f(x1,x2)=(75*x1 + 75*x2)*8/500+ (100*x1 + 50*x2)*6/500 -(3*x1 + 2*x2))}

- Restricciones:

    - 75x1 + 75x2 <= 4000 
    - 100x1+ 50x2 <= 5000
    - x1 -2X2 <= 0 




In [36]:
# Recursos
import dimod
from dimod import ConstrainedQuadraticModel, ExactCQMSolver, Integer
from dwave.system import DWaveSampler, EmbeddingComposite

# Funciones auxiliares

def sol_factibles(sampleset,n=None):
    # Resultados
    
    subset=sampleset.filter(lambda s: s.is_feasible).aggregate()
    
    if not n:
        print(f"\nLas {len(subset)} soluciones factibles al problema son:\n")
    else:
        print(f"\nImprimiendo las {n}/{len(subset)} primeras soluciones factibles:\n")
    
    print(subset.slice(n))
    
    
def sol_problema(result):
    # Agregación de los n reads
    samples = []
    ocurrencias = []

    for s in result.data():
        samples.append(invert(s.sample))
        ocurrencias.append(s.num_occurrences)

    sampleset = dimod.SampleSet.from_samples_cqm(samples,cqm,num_occurrences=ocurrencias)
    return(sampleset)

#### Caso simple

Obtener dimensionado surtidos atendiendo al criterio de maximizar el importe de venta 

In [22]:
## Definición del problema CQM

x1 = Integer('Xa',upper_bound=40)
x2 = Integer('Xb',upper_bound=40)

cqm = ConstrainedQuadraticModel()


# Función objetivo: Maximizar (ventas-costes)o minimizar (-ventas+costes)

cqm.set_objective(-3*x1 - 2*x2)

# Restricciones

cqm.add_constraint(75*x1 + 75*x2 <= 4000,"Producción 1: 75*x1 + 75*x2 <= 4000")
cqm.add_constraint(100*x1 + 50*x2 <= 5000,"Producción 2: 100*x1 + 50*x2 <= 5000")
cqm.add_constraint(x1 - 2*x2 <= 0,"Producción 3: x1 - 2*x2 <= 0")
print("Planteamiento problema CQM")
print("==========================")
print(cqm)

Planteamiento problema CQM
Constrained quadratic model: 2 variables, 3 constraints, 8 biases

Objective
  -3*Integer('Xa') - 2*Integer('Xb')

Constraints
  Producción 1: 75*x1 + 75*x2 <= 4000: 75*Integer('Xa') + 75*Integer('Xb') <= 4000.0
  Producción 2: 100*x1 + 50*x2 <= 5000: 100*Integer('Xa') + 50*Integer('Xb') <= 5000.0
  Producción 3: x1 - 2*x2 <= 0: Integer('Xa') - 2*Integer('Xb') <= 0.0

Bounds
  0.0 <= Integer('Xa') <= 40.0
  0.0 <= Integer('Xb') <= 40.0



### Solución por fuerza bruta

In [24]:
result = ExactCQMSolver().sample_cqm(cqm)

In [25]:
solCPU=sol_factibles(result,10)


Imprimiendo las 10/899 primeras soluciones factibles:

  Xa Xb energy num_oc. is_sat. is_fea.
0 35 18 -141.0       1 arra...    True
1 34 19 -140.0       1 arra...    True
2 33 20 -139.0       1 arra...    True
3 32 21 -138.0       1 arra...    True
4 34 18 -138.0       1 arra...    True
5 33 19 -137.0       1 arra...    True
6 31 22 -137.0       1 arra...    True
7 34 17 -136.0       1 arra...    True
8 32 20 -136.0       1 arra...    True
9 30 23 -136.0       1 arra...    True
['INTEGER', 10 rows, 10 samples, 2 variables]



<b>Resultados:</b>

- Lotes: A: 35  B: 18

- Importe ventas: 141€

- Costes: 

    - Arándanos: 75x35 + 75x18 = 3975,   coste: 63.6€
    - Frambuesas: 100x35 + 50x18 = 4400, coste: 52.8€
    
- Beneficio: 

    - Ventas-coste: 141-63.6-52.8= 24.6€
    


#### Función objetivo: minimizar costes-ventas

Se va ahora a incluir en la funcón objetivo el coste de los lotes, evitando así el posprocesado numérico para obtener el beneficio.

Maximizar (ventas-costes) equivale a  Minimizar (-ventas+costes)

In [26]:
## Definición del problema CQM

x1 = Integer('Xa',upper_bound=40)
x2 = Integer('Xb',upper_bound=40)

cqm = ConstrainedQuadraticModel()


# Función objetivo: 

cqm.set_objective((75*x1 + 75*x2)*8/500+ (100*x1 + 50*x2)*6/500 -(3*x1 + 2*x2))

# Restricciones

cqm.add_constraint(75*x1 + 75*x2 <= 4000,"Producción 1: 75*x1 + 75*x2 <= 4000")
cqm.add_constraint(100*x1 + 50*x2 <= 5000,"Producción 2: 100*x1 + 50*x2 <= 5000")
cqm.add_constraint(x1 - 2*x2 <= 0,"Producción 3: x1 - 2*x2 <= 0")
print("Planteamiento problema CQM")
print("==========================")
print(cqm)

Planteamiento problema CQM
Constrained quadratic model: 2 variables, 3 constraints, 8 biases

Objective
  -0.6000000000000001*Integer('Xa') - 0.20000000000000018*Integer('Xb')

Constraints
  Producción 1: 75*x1 + 75*x2 <= 4000: 75*Integer('Xa') + 75*Integer('Xb') <= 4000.0
  Producción 2: 100*x1 + 50*x2 <= 5000: 100*Integer('Xa') + 50*Integer('Xb') <= 5000.0
  Producción 3: x1 - 2*x2 <= 0: Integer('Xa') - 2*Integer('Xb') <= 0.0

Bounds
  0.0 <= Integer('Xa') <= 40.0
  0.0 <= Integer('Xb') <= 40.0



In [27]:
result = ExactCQMSolver().sample_cqm(cqm)

In [28]:
solCPU=sol_factibles(result,10)


Imprimiendo las 10/899 primeras soluciones factibles:

  Xa Xb energy num_oc. is_sat. is_fea.
0 35 18  -24.6       1 arra...    True
1 34 19  -24.2       1 arra...    True
2 34 18  -24.0       1 arra...    True
3 33 20  -23.8       1 arra...    True
4 34 17  -23.8       1 arra...    True
5 33 19  -23.6       1 arra...    True
6 33 18  -23.4       1 arra...    True
7 32 21  -23.4       1 arra...    True
8 33 17  -23.2       1 arra...    True
9 32 20  -23.2       1 arra...    True
['INTEGER', 10 rows, 10 samples, 2 variables]


### Solución con QPU

In [33]:
#reformulación qubo y sampling

fl=20 # factor penalizador de lagrange

qubo, invert = dimod.cqm_to_bqm(cqm, lagrange_multiplier = fl)

sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(qubo, num_reads=2000)

In [34]:
sampleset=sol_problema(result)
solQPU=sol_factibles(sampleset,5)


Imprimiendo las 5/384 primeras soluciones factibles:

  Xa Xb energy num_oc. is_sat. is_fea.
0 35 18  -24.6       1 arra...    True
1 34 19  -24.2       1 arra...    True
2 33 20  -23.8       2 arra...    True
3 33 19  -23.6       1 arra...    True
4 33 18  -23.4       1 arra...    True
['INTEGER', 5 rows, 6 samples, 2 variables]


### <b>Analizando los resultados del sampler QPU</b>

- Se ha formulado un problema Qubo desde un problema CQM

- La fuerza bruta es útil para estimar el upper_bound del espacio de entradas, para luego ajustarlo para la QPU

- El espacio de soluciones factibles es muy inferior al de fuerza bruta

- Se han replicado los valores óptimos obtenidos por fuerza bruta

- Con 1000 samples se resolvió el problema con función de coste simple

- Hubo que subir a 2000 para obtener el beneficio correcto con una única ocurrencia.

